In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import datatable as dt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

'''
raw_train =  dt.fread("../input/tabular-playground-series-sep21-processed-data/Tabular Playground_Series_Sep_2021_train_processed_data.csv").to_pandas()
raw_test = dt.fread("../input/tabular-playground-series-sep21-processed-data/Tabular Playground_Series_Sep_2021_test_processed_data.csv").to_pandas()
sample= dt.fread("../input/tabular-playground-series-sep-2021/sample_solution.csv").to_pandas()

'''

'''
raw_train =  dt.fread("../input/tabular-playground-series-sep-2021/train.csv").to_pandas()
raw_test = dt.fread("../input/tabular-playground-series-sep-2021/test.csv").to_pandas()
sample= dt.fread("../input/tabular-playground-series-sep-2021/sample_solution.csv").to_pandas()

'''
raw_train =  dt.fread("../input/tps-sep2021-dataset/train_filled with dictionary.csv").to_pandas()
raw_test = dt.fread("../input/tps-sep2021-dataset/test_filled with dictionary.csv").to_pandas()
sample= dt.fread("../input/tabular-playground-series-sep-2021/sample_solution.csv").to_pandas()

# Making a copy of raw data
train = raw_train.copy()
test = raw_test.copy()

# Shape
print("train shape :" , train.shape)
print("test shape :" , test.shape)



# Split target and remove de id column  in both dataset

'''
train.drop(['id','claim','n_missing','C0'], axis = 1, inplace = True)
test.drop(['id','n_missing','C0'], axis = 1, inplace = True)
'''

'''

train.drop(['id','claim'], axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)
'''


In [ ]:
target = raw_train.claim

In [ ]:
target

In [ ]:
train.drop(['id','claim'], axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)

In [ ]:
train.columns

In [ ]:
test

In [ ]:
X = train
X_test = test
y = target

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
scaler = RobustScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.impute import SimpleImputer

features = train.columns

train['n_missing'] = train[features].isna().sum(axis=1)
test['n_missing'] = test[features].isna().sum(axis=1)

train['std'] = train[features].std(axis=1)
test['std'] = test[features].std(axis=1)

si = SimpleImputer(strategy ='mean')
train[features] = si.fit_transform(train[features])
test[features] = si.transform(test[features])



In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

params = {   
               'objective': 'binary',
               'boosting_type': 'gbdt', #gbdt
               'num_leaves': 6, #6 #2^(max_depth)
               'max_depth': 2, #2  
               'learning_rate': 0.5, #0.1
               'n_estimators': 40000, #40000
               'reg_alpha': 25.0,
               'reg_lambda': 76.7,
               'bagging_seed': 7014, #42
               'feature_fraction_seed': 7014, #42
               'n_jobs': 4,
               'subsample': 0.98,
               'subsample_freq': 1,
               'colsample_bytree': 0.69,
               'min_child_samples': 54,
               'min_child_weight': 256,
                #'drop_seed':int(2**n_fold)
}

preds_valid_f = {}
preds_test = []
total_auc = []

kf = KFold(n_splits=5,random_state=7014,shuffle=True)

for fold,(train_index, valid_index) in enumerate(kf.split(train,target)):

    X_train,X_valid = train.loc[train_index], train.loc[valid_index]
    y_train,y_valid = target.loc[train_index], target.loc[valid_index]
    
    # Preprocessing
    index_valid  = X_valid.index.tolist()
    #--------------------------------------------------------
    # model
    model = lgb.LGBMClassifier(**params,random_state = 7014)
    model.fit(X_train, y_train,
              verbose=False,
              # These three parameters will stop training before a model starts overfitting 
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric="auc",
              early_stopping_rounds=300,
              )
    # oof
    preds_valid = model.predict_proba(X_valid)[:,1]
    #--------------------------------------------------------
    preds_test.append(model.predict_proba(test)[:,1])
    #--------------------------------------------------------
    preds_valid_f.update(dict(zip(index_valid, preds_valid)))

    # Getting score for a fold model
    fold_auc = roc_auc_score(y_valid, preds_valid)
    print(f"Fold {fold} roc_auc_score: {fold_auc}")
    # Total rmse
    total_auc.append(fold_auc)
    
print(f"mean roc_auc_score: {np.mean(total_auc)}, std: {np.std(total_auc)}")

In [ ]:
output = pd.DataFrame({'id': raw_test.id,
                       'claim': np.mean(preds_test, axis = 0)})
output.to_csv('preds.csv', index=False)
output.head()